In [1]:
import torch
import torchvision.models as models 
from torch.profiler import profile, record_function, ProfilerActivity
import numpy as np 
from scipy.stats import random_correlation 
from scipy.stats import multivariate_normal as normal 
import datetime as dt
import matplotlib.pyplot as plt
import time
####################################### Base parameters #######################################
X_0_sc = 0        ## initial X scalar
T=1               ## time horizon
N=100             ## number of discretization for time horizon T
B_0_sc= 1         ## drift scalar baseline
Sig_0_sc= 1       ## vol scalar baseline 
DIM_arr = np.array([1, 5, 10, 20, 50, 100])
Num_Iter = 10
####################################### MC iterations #######################################
M   = 3000000            ## number of outer mc samples  (M_0>1)
M_w = 30000              ## number of outer mc samples  (M_1<M_0)
mps_device = torch.device("mps")
V0_array = torch.zeros([Num_Iter,DIM_arr.shape[0]], device="mps")
W0_array = torch.zeros([Num_Iter,DIM_arr.shape[0],2], device="mps")
Time_array = torch.zeros([Num_Iter,DIM_arr.shape[0]], device="mps")
C_sig_array = torch.zeros([Num_Iter,DIM_arr.shape[0]], device="mps")
b_norm_array = torch.zeros([Num_Iter,DIM_arr.shape[0]], device="mps")
b_norm_array_2 = torch.zeros([Num_Iter,DIM_arr.shape[0]], device="mps")
for k in range(DIM_arr.shape[0]):
    DIM = DIM_arr[k]
    if DIM == 1:
        B_0 = np.float32(np.ones((1,1))*B_0_sc)
        Sig_0 = np.float32(np.ones((1,1))*Sig_0_sc)
        X_0= np.float32(np.ones((1,1))*X_0_sc)
    else:
        B_00 = np.random.uniform(0,1,(DIM))
        B_0 = np.float32(B_00/np.sum(B_00)) 
        Sig_00 = np.random.uniform(-1,1,(DIM,DIM))
        Sig_0 = np.float32(Sig_00 / np.linalg.norm(np.sum(Sig_00, axis=0)))
        X_0 = np.float32(np.ones(DIM)*X_0_sc)
    C_sig_array[0,k] = torch.tensor(np.sqrt(np.amin(np.linalg.eig(np.matmul(np.transpose(Sig_0),Sig_0))[0])), device="mps")
    b_norm_array[0,k] = torch.tensor(np.sum(B_0), device="mps")
    b_norm_array_2[0,k] = torch.tensor(np.linalg.norm(B_0), device="mps")
    print(Sig_0)
    for j in range(Num_Iter):
        start = time.time()
        with torch.device(mps_device): 
            X_0_tor = torch.tensor(torch.from_numpy(X_0), device="mps") 
            B_0_tor = torch.tensor(torch.from_numpy(B_0), device="mps") 
            Sig_0_tor = torch.tensor(torch.from_numpy(Sig_0), device="mps") 
            # ##############################     Float increment     ##############################
            Dt_tor = torch.tensor(T/N) 
            DIM_tor = torch.tensor(DIM) 
            # ##############################     Random sample generation     ##############################
            T_arr_tor= torch.arange(1,N+1)*Dt_tor 
            R_T_arr_tor=torch.linspace(T,0,N+1)[:-1] 
            Sqrt_T_arr_tor = torch.sqrt(T_arr_tor) 
            R_Sqrt_T_arr_tor = torch.sqrt(R_T_arr_tor) 
            W_sample = torch.tensor(torch.FloatTensor(DIM, M).normal_(), device="mps") 
            Dum_O1, Dum_O2, Dum_O3, Dum_O4 = torch.ones([M_w,DIM,DIM]), torch.ones([M_w, N, DIM]), torch.ones([M_w, DIM, N]), torch.ones([N, DIM, DIM])
            ############################
            Dum_O5 = torch.ones([M,DIM])
            ############################
            Dum_Z1, Dum_Z2 = torch.zeros([DIM,DIM]), torch.zeros([M_w,DIM,DIM])
            V1_w = torch.zeros(1)
            V1_Jac_w = torch.zeros(1)
        R_Sqrt_T_arr_tensor=torch.permute(Dum_O3*R_Sqrt_T_arr_tor, (2,0,1))
        Sqrt_T_arr_tensor=torch.permute(Dum_O3*Sqrt_T_arr_tor, (2,0,1))
        X0_D = Dum_O2 * X_0_tor 
        DR_D = Dum_O2 * B_0_tor 
        DIF_D1 = torch.permute(R_Sqrt_T_arr_tensor*torch.transpose(torch.matmul(Sig_0_tor,W_sample[:,:M_w]),0,1), (1,0,2))
        DIF_D2 = torch.permute(Sqrt_T_arr_tensor*torch.transpose(torch.matmul(Sig_0_tor,W_sample[:,:M_w]),0,1), (1,0,2))
        #########################
        X0_D_V = Dum_O5 * X_0_tor 
        DR_D_V = Dum_O5 * B_0_tor
        def func_v(X_T):  
            inner_sum = torch.sum(X_T, dim=1)
            v_value = torch.mean(torch.sin(inner_sum))               
            return v_value    
        XT_V = X0_D_V + DR_D_V * T_arr_tor[-1] + torch.transpose(torch.matmul(Sig_0_tor,W_sample),0,1) * Sqrt_T_arr_tor[-1]   #### num of disc, num of MC, dim of X
        J_w_const_mtx = torch.permute(torch.matmul(Dum_O4,Sig_0_tor),[1,2,0])
        def func_w(X_T):   
            inner_sum = torch.sum(X_T, dim=2) 
            w_value = torch.mean(torch.cos(inner_sum),dim=0)                ### [N]
            w_value_d = Dum_O3[0,:,:] * w_value                             ### [DIM, N]
            J_w_value = torch.mean(torch.sin(inner_sum),dim=0)              
            J_w_value_mtx =  - J_w_const_mtx* J_w_value                     ### [DIM, DIM, N]  
            norm_w, norm_J_w_sig_0 = torch.linalg.norm(w_value_d, dim=0), torch.linalg.norm(J_w_value_mtx,dim=(0,1))
            return norm_w, norm_J_w_sig_0, torch.sum(norm_w), torch.sum(norm_J_w_sig_0)
        XT_W_FIX =  X0_D + DR_D * T_arr_tor[-1] + DIF_D1
        XT_W_FIX.shape

        V0_array[j,k] = func_v(XT_V)
        pc_M = 0
        p=1
        for l in range(M_w):
            V1_Temp = func_w(XT_W_FIX + Dum_O2 * DIF_D2[l,:,:])
            V1_w += V1_Temp[2]
            V1_Jac_w += V1_Temp[3]
            if (pc_M == 10000):
                print('%7.d MC-paths' %(p*10000)) 
                p+=1 
                pc_M = 0 
            pc_M +=1 
        W0_array[j,k,:] = torch.tensor([V1_w * Dt_tor / M_w , V1_Jac_w * Dt_tor / M_w], device="mps") 
        end = time.time() 
        Time_array[j,k]= torch.tensor(end - start , device="mps")
####################################### Save #######################################
V0_array_np =  torch.Tensor.numpy(torch.as_tensor(V0_array, dtype=torch.float32, device='cpu'))
W0_array_np =  torch.Tensor.numpy(torch.as_tensor(W0_array, dtype=torch.float32, device='cpu'))
Time_array_np =  torch.Tensor.numpy(torch.as_tensor(Time_array, dtype=torch.float32, device='cpu'))
C_sig_array_np = torch.Tensor.numpy(torch.as_tensor(C_sig_array, dtype=torch.float32, device='cpu'))
b_norm_array_np = torch.Tensor.numpy(torch.as_tensor(b_norm_array, dtype=torch.float32, device='cpu'))
b_norm_array_np_2 = torch.Tensor.numpy(torch.as_tensor(b_norm_array_2, dtype=torch.float32, device='cpu'))
np.savetxt('V0.csv', V0_array_np, delimiter=",")
np.savetxt('W0_drift.csv', W0_array_np[:,:,0], delimiter=",")
np.savetxt('W0_vol.csv', W0_array_np[:,:,1], delimiter=",")
np.savetxt('Time.csv', Time_array_np, delimiter=",")
np.savetxt('C_sig.csv', C_sig_array_np, delimiter=",")
np.savetxt('b_norm.csv', b_norm_array_np, delimiter=",")

[[1.]]


/Users/khpark/anaconda3/lib/python3.11/site-packages/torch/utils/_device.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


  10000 MC-paths


KeyboardInterrupt: 